In [12]:
# Imports
import pandas as pd
import requests, io
import hvplot.pandas
import panel as pn
pn.extension()

print("Libraries loaded!")


Libraries loaded!


In [ ]:
# Create basin dict with NOAA division IDs
basins = {
    "Snake River Basin": "4802",
    "Green River Basin": "4804",
    "Wind River Basin": "4805",
}
print('Basin dictionary created!')
basins.keys()

Basin dictionary created!


dict_keys(['Snake River Basin', 'Green River Basin', 'Wind River Basin'])

In [ ]:
# Load PDSI data for each basin
data_dict = {}
for name, div_id in basins.items():
    try:
        url = f"https://www.ncei.noaa.gov/access/monitoring/climate-at-a-glance/divisional/time-series/{div_id}/pdsi/1/5/1995-2025.csv"
        response = requests.get(url, headers={"User-Agent": "Mozilla/5.0"})
        response.raise_for_status()
        
        df = pd.read_csv(io.StringIO(response.text), skiprows=2)
        df = df.rename(columns={df.columns[0]: "Date", df.columns[1]: "PDSI"})
        df["Date"] = pd.to_datetime(df["Date"], format="%Y%m", errors="coerce")
        df = df.dropna(subset=["Date"])
        data_dict[name] = df
        print(f"Loaded {name}: {len(df)} rows")
        
    except Exception as e:
        print(f"Failed to load {name}: {e}")
        data_dict[name] = pd.DataFrame(columns=["Date", "PDSI"])

Loaded Snake River Basin: 30 rows
Loaded Green River Basin: 30 rows
Loaded Wind River Basin: 30 rows


In [16]:
for name, df in data_dict.items():
    print(name, len(df))
    print(df.head())



Snake River Basin 30
        Date  PDSI
0 1996-05-01  4.39
1 1997-05-01  3.71
2 1998-05-01  2.71
3 1999-05-01  2.84
4 2000-05-01 -1.36
Green River Basin 30
        Date  PDSI
0 1996-05-01  4.46
1 1997-05-01  1.50
2 1998-05-01  2.15
3 1999-05-01  3.07
4 2000-05-01 -2.56
Wind River Basin 30
        Date  PDSI
0 1996-05-01  4.60
1 1997-05-01  1.86
2 1998-05-01 -1.38
3 1999-05-01  3.05
4 2000-05-01 -1.80


In [ ]:
# Panel dropdown for basin selection
basin_selector = pn.widgets.Select(name="Choose Basin", options=list(data_dict.keys()))


In [7]:
''' 
PDSI is calculated by climate division, not HUC
Snake River Basin: Mapped to Climate Division 2 (Northwest)

Green River Basin: Mapped to Climate Division 4 (Southwest)

Wind River Basin: Mapped to Climate Division 5 (Central)
'''

' \nPDSI is calculated by climate division, not HUC\nSnake River Basin: Mapped to Climate Division 2 (Northwest)\n\nGreen River Basin: Mapped to Climate Division 4 (Southwest)\n\nWind River Basin: Mapped to Climate Division 5 (Central)\n'

In [19]:
# Create interactive plot 
@pn.depends(basin_selector.param.value)
def plot_pdsi(basin_name):
    df = data_dict[basin_name]
    if df.empty:
        return pn.pane.Markdown(f"No data available for {basin_name}.")
    return df.hvplot.line(
        x="Date", y="PDSI",
        title=f"PDSI for {basin_name}",
        ylabel="PDSI",
        xlabel="Date",
        width=800, height=400
    )




In [ ]:
# Create dashboard
dashboard = pn.Column(
    "# Interactive PDSI Dashboard",
    basin_selector,
    plot_pdsi
)

# Call dashboard
dashboard